# **MIP2-Run1**

In [2]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [3]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [5]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [5]:
problem = """You are the production planner in a food and beverage production fascility. You are tasked with planning production for the next 12 time periods with the goal of maximizing profit. The production fascility is able to produce two types of food and three types of beverage. The food is produced in whole units, while the beverage can be produced in any fraction of a liter. For each product that is produced at period t, a fixed cost and variable cost incurred. Each product can also be stored which is also associated with a variable cost per product stored. There is also a maximum demand for each product and time period which means that a specific product will not be sold anymore if the demand is met for that time period. Please note, that the demand does not have to be met. The revenue per product and time also differs. The cost for production and storage, the demand and revenue are all provided as in sepearte csv files named "fixed_cost_production.csv", "variable_cost_production.csv", "variable_cost_storage.csv", "demand.csv" and "revenue.csv" where all files follow the structre of the columns representing time and rows representing product. Finally, there is a constant storage capacity and production capacity for each product that resets each time period. The storage capacity is 580 units of food 1, 687 units of food 2, 599L of beverage one, 788 Liters of beverage 2 and 294L of beverage 3. The production capacity is 1080 units of food 1, 908 units of food 2, 408L of beverage one, 1000L of beverage 2 and 403L of beverage 3."""

In [6]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [7]:
print(response1.choices[0].message.content)

**Parameters:**

Let's denote products as p where p ∈ {1,2,3...,5} representing food 1, food 2, beverage 1, beverage 2 and beverage 3 respectively and a time period denoted by t where t ∈ {1,2,3,...,12}.

- Fixed cost of production: fc[p,t]
- Variable cost of production: vc[p,t]
- Variable cost of storage: sc[p,t]
- Demand: d[p,t]
- Revenue: r[p,t]
- Maximum storage capacity per product: ms[p] = {580, 687, 599, 788, 294} for each product p.
- Maximum production capacity per product: mp[p] = {1080, 908, 408, 1000, 403} for each product p.

**Decision Variables:**

- Quantity to manufacture of each product every period: X[p,t] >= 0, where X stands for units for food and liters for beverages.
- Quantity to store for each product every period: Y[p,t] >= 0, where Y stands for units for food and litres for beverages.

**Objective Function:**

The objective of the problem is to maximize total profit, defined as total revenue minus total costs (both production and storage costs).

maximize ∑(p

### **Generate Pyomo Code**

In [8]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [9]:
Markdown(response2.choices[0].message.content)

Here is the Pyomo implementation for this optimization problem:

```python
from pyomo.environ import *

# Sample data
products = range(1,6)
time_periods = range(1,13)

fc = {(p,t):5 for p in products for t in time_periods}  # Fixed cost of production
vc = {(p,t):10 for p in products for t in time_periods}  # Variable cost of production
sc = {(p,t):2 for p in products for t in time_periods}   # Variable cost of storage
d = {(p,t):20 for p in products for t in time_periods}  # Demand
r = {(p,t):15 for p in products for t in time_periods}   # Revenue
mp = {p:1000 for p in products}    # Maximum production capacity per product
ms = {p:500 for p in products}    # Maximum storage capacity per product

# Model
model = ConcreteModel()

# Indexes
model.products = Set(initialize=products)
model.time_periods = Set(initialize=time_periods)

# Variables
model.X = Var(model.products, model.time_periods, within=NonNegativeReals)
model.Y = Var(model.products, model.time_periods, within=NonNegativeReals)
model.binary_var = Var(model.products,model.time_periods, within=Binary)

# Objective function
def obj_rule(m):
    return sum((r[p,t]*min(d[p,t], m.X[p,t]) - (vc[p,t]*m.X[p,t] + fc[p,t]*m.binary_var[p,t] + sc[p,t]*m.Y[p,t])) for p in m.products for t in m.time_periods)
model.obj = Objective(rule=obj_rule, sense=maximize)

# Constraints

def production_capacity_rule(m, p, t):
   return m.X[p,t] <= mp[p]
model.production_capacity = Constraint(model.products,model.time_periods, rule=production_capacity_rule)

def storage_capacity_rule(m, p, t):
   return m.Y[p,t] <= ms[p]
model.storage_capacity = Constraint(model.products,model.time_periods, rule=storage_capacity_rule)

def initial_storage_rule(m, p):
   return m.Y[p,1] <= m.X[p,1] - min(d[p,1], m.X[p,1])
model.initial_storage = Constraint(model.products, rule=initial_storage_rule)

def flow_conservation_rule(m, p, t):
    if t>1:
        return m.Y[p,t] == m.Y[p,t-1] + m.X[p,t] - min(d[p,t], m.X[p,t] + m.Y[p,t-1])
    else:
        return Constraint.Skip
model.flow_conservation = Constraint(model.products,model.time_periods, rule=flow_conservation_rule)

def binary_var_rule(m, p, t):
    return m.binary_var[p,t] <= m.X[p,t]
model.binary_var = Constraint(model.products,model.time_periods, rule=binary_var_rule)
```

### **Run the code Generated by GPT4**

In [6]:
from pyomo.environ import *

# Sample data
products = range(1,6)
time_periods = range(1,13)

fc = {(p,t):5 for p in products for t in time_periods}  # Fixed cost of production
vc = {(p,t):10 for p in products for t in time_periods}  # Variable cost of production
sc = {(p,t):2 for p in products for t in time_periods}   # Variable cost of storage
d = {(p,t):20 for p in products for t in time_periods}  # Demand
r = {(p,t):15 for p in products for t in time_periods}   # Revenue
mp = {p:1000 for p in products}    # Maximum production capacity per product
ms = {p:500 for p in products}    # Maximum storage capacity per product

# Model
model = ConcreteModel()

# Indexes
model.products = Set(initialize=products)
model.time_periods = Set(initialize=time_periods)

# Variables
model.X = Var(model.products, model.time_periods, within=NonNegativeReals)
model.Y = Var(model.products, model.time_periods, within=NonNegativeReals)
model.binary_var = Var(model.products,model.time_periods, within=Binary)

# Objective function
def obj_rule(m):
    return sum((r[p,t]*min(d[p,t], m.X[p,t]) - (vc[p,t]*m.X[p,t] + fc[p,t]*m.binary_var[p,t] + sc[p,t]*m.Y[p,t])) for p in m.products for t in m.time_periods)
model.obj = Objective(rule=obj_rule, sense=maximize)

# Constraints

def production_capacity_rule(m, p, t):
   return m.X[p,t] <= mp[p]
model.production_capacity = Constraint(model.products,model.time_periods, rule=production_capacity_rule)

def storage_capacity_rule(m, p, t):
   return m.Y[p,t] <= ms[p]
model.storage_capacity = Constraint(model.products,model.time_periods, rule=storage_capacity_rule)

def initial_storage_rule(m, p):
   return m.Y[p,1] <= m.X[p,1] - min(d[p,1], m.X[p,1])
model.initial_storage = Constraint(model.products, rule=initial_storage_rule)

def flow_conservation_rule(m, p, t):
    if t>1:
        return m.Y[p,t] == m.Y[p,t-1] + m.X[p,t] - min(d[p,t], m.X[p,t] + m.Y[p,t-1])
    else:
        return Constraint.Skip
model.flow_conservation = Constraint(model.products,model.time_periods, rule=flow_conservation_rule)

def binary_var_rule(m, p, t):
    return m.binary_var[p,t] <= m.X[p,t]
model.binary_var = Constraint(model.products,model.time_periods, rule=binary_var_rule)

ERROR:pyomo.core:Rule failed when generating expression for Objective obj with index None:
PyomoException: Cannot convert non-constant Pyomo expression (X[1,1]  <  20) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.
ERROR:pyomo.core:Constructing component 'obj' from data=None failed:
    PyomoException: Cannot convert non-constant Pyomo expression (X[1,1]  <  20) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both ca

PyomoException: Cannot convert non-constant Pyomo expression (X[1,1]  <  20) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**